# Partie 3 : Les modules DSPy

In [5]:
import dspy
import warnings
warnings.filterwarnings('ignore')

# Configurer le modèle de langage
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

# Configurer DSPy globalement
dspy.configure(lm=lm)

# Tester la configuration
response = lm("Bonjour, ça fonctionne ?")
print(response)

["Bonjour ! Oui, tout semble fonctionner correctement. Comment puis-je vous aider aujourd'hui ?"]


In [6]:
# Catégories et priorités possibles
CATEGORIES = ["Hardware", "Software", "Network", "Application", "Infrastructure", "Account", "Email", "Peripherals"]
PRIORITIES = ["Low", "Medium", "High", "Urgent", "Critical"]

class TicketClassifier(dspy.Signature):
    """Classifier un ticket de support IT selon sa catégorie et sa priorité."""
    
    ticket = dspy.InputField(desc="Description du ticket de support IT")
    category = dspy.OutputField(desc=f"Catégorie parmi: {', '.join(CATEGORIES)}")
    priority = dspy.OutputField(desc=f"Priorité parmi: {', '.join(PRIORITIES)}")

## Qu'est-ce qu'un module ?

Un **module** dans DSPy est un composant qui **utilise une signature** pour générer des prédictions.

### Analogie
- **Signature** = Le contrat ("je te donne X, tu me donnes Y")
- **Module** = L'employé qui exécute le contrat (avec sa propre méthode de travail)

### Les différents types de modules

DSPy offre plusieurs modules, chacun avec une stratégie différente :

1. **Predict** : Génération directe (le plus simple)
2. **ChainOfThought** : Raisonnement avant de répondre
3. **ReAct** : Raisonnement avec actions possibles
4. **ProgramOfThought** : Génération de code pour raisonner
5. **Modules personnalisés** : Composition de plusieurs modules

## Module 1 : Predict (le plus simple)

**Predict** est le module de base : il génère directement une réponse.

### Fonctionnement
1. Reçoit les entrées
2. Génère immédiatement les sorties
3. Retourne le résultat

### Quand l'utiliser
- Tâches simples
- Besoin de rapidité
- Première version d'un système

In [7]:
# Créer un module Predict avec notre signature
predict_classifier = dspy.Predict(TicketClassifier)

# Tester sur un exemple
test_ticket = "Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure."
result = predict_classifier(ticket=test_ticket)

print("🔮 Module : Predict")
print(f"📝 Ticket : {test_ticket}")
print(f"📦 Catégorie prédite : {result.category}")
print(f"⚡ Priorité prédite : {result.priority}")

🔮 Module : Predict
📝 Ticket : Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure.
📦 Catégorie prédite : Hardware
⚡ Priorité prédite : Urgent


## Module 2 : ChainOfThought (avec raisonnement)

**ChainOfThought** demande au modèle de raisonner avant de répondre.

### Fonctionnement
1. Reçoit les entrées
2. **Génère d'abord un raisonnement**
3. Génère ensuite les sorties basées sur ce raisonnement
4. Retourne le résultat (avec le raisonnement)

### Quand l'utiliser
- Tâches complexes nécessitant de la réflexion
- Besoin d'expliquer les décisions
- Améliorer la précision (+5-15% typiquement)

### Avantages
- ✅ Meilleure précision
- ✅ Raisonnement inspectable
- ✅ Plus robuste sur des cas complexes

In [8]:
# Créer un module ChainOfThought avec notre signature
cot_classifier = dspy.ChainOfThought(TicketClassifier)

# Tester sur le même exemple
result = cot_classifier(ticket=test_ticket)

print("🧠 Module : ChainOfThought")
print(f"📝 Ticket : {test_ticket}")
print(f"💭 Raisonnement : {getattr(result, 'rationale', 'non disponible')}")
print(f"📦 Catégorie prédite : {result.category}")
print(f"⚡ Priorité prédite : {result.priority}")

🧠 Module : ChainOfThought
📝 Ticket : Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure.
💭 Raisonnement : non disponible
📦 Catégorie prédite : Hardware
⚡ Priorité prédite : High


### Comparaison Predict vs ChainOfThought

Testons les deux modules sur plusieurs exemples pour voir la différence.

In [9]:
# Données de validation
valset = [
    {"ticket": "Le serveur de fichiers est inaccessible. Personne ne peut travailler.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.", "category": "Account", "priority": "Urgent"},
    {"ticket": "L'écran de mon collègue en vacances clignote. On peut attendre.", "category": "Hardware", "priority": "Low"},
    {"ticket": "Le CRM plante quand j'essaie d'exporter les contacts.", "category": "Application", "priority": "High"},
    {"ticket": "Je voudrais changer ma photo de profil quand vous aurez un moment.", "category": "Account", "priority": "Low"},
    {"ticket": "La vidéoconférence ne fonctionne pas. Réunion avec New York dans 10 minutes!", "category": "Application", "priority": "Critical"},
    {"ticket": "Mon antivirus affiche un message d'expiration mais tout fonctionne.", "category": "Software", "priority": "Medium"}
]

# Tester sur 3 exemples
test_cases = valset[:3]

print("="*70)
print("Comparaison Predict vs ChainOfThought")
print("="*70 + "\n")

for i, example in enumerate(test_cases, 1):
    print(f"--- Exemple {i} ---")
    print(f"Ticket : {example['ticket']}")
    print(f"Attendu : {example['category']} | {example['priority']}\n")
    
    # Predict
    pred_result = predict_classifier(ticket=example['ticket'])
    print(f"  Predict : {pred_result.category} | {pred_result.priority}")
    
    # ChainOfThought
    cot_result = cot_classifier(ticket=example['ticket'])
    print(f"  ChainOfThought : {cot_result.category} | {cot_result.priority}")
    print()

Comparaison Predict vs ChainOfThought

--- Exemple 1 ---
Ticket : Le serveur de fichiers est inaccessible. Personne ne peut travailler.
Attendu : Infrastructure | Critical

  Predict : Infrastructure | High
  ChainOfThought : Infrastructure | High

--- Exemple 2 ---
Ticket : J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.
Attendu : Account | Urgent

  Predict : Account | Urgent
  ChainOfThought : Account | Urgent

--- Exemple 3 ---
Ticket : L'écran de mon collègue en vacances clignote. On peut attendre.
Attendu : Hardware | Low

  Predict : Hardware | Low
  ChainOfThought : Hardware | Low



## Module 3 : ReAct (raisonnement + actions)

**ReAct** alterne entre raisonnement et actions. C'est un module qui permet au modèle d'interagir avec des outils externes pour résoudre des problèmes.

### Fonctionnement

1. **Raisonne** sur le problème
2. **Décide** d'une action à faire (ex: chercher dans une base de données)
3. **Observe** le résultat de l'action
4. **Raisonne** à nouveau avec cette nouvelle information
5. **Répète** jusqu'à avoir la réponse

### Quand l'utiliser

- Besoin d'interactions avec des outils externes
- Recherche d'informations nécessaire
- Tâches multi-étapes nécessitant des actions

### Note importante

ReAct nécessite de définir des **outils** (fonctions) que le modèle peut appeler. Le modèle décide quand et comment utiliser ces outils.

### Exemple : ReAct avec base de données de tickets

Simulons une base de données de tickets historiques pour démontrer ReAct.

In [ ]:
# Base de données simulée de tickets similaires
ticket_database = {
    "hardware_boot": [
        {"ticket": "PC ne démarre pas", "solution": "Vérifier l'alimentation", "resolution_time": "2h"},
        {"ticket": "Ordinateur bloqué au démarrage", "solution": "Mode sans échec + réparation", "resolution_time": "1h"}
    ],
    "network_vpn": [
        {"ticket": "VPN instable", "solution": "Mise à jour client VPN", "resolution_time": "30min"},
        {"ticket": "Déconnexions VPN fréquentes", "solution": "Changer serveur VPN", "resolution_time": "15min"}
    ],
    "printer": [
        {"ticket": "Imprimante hors ligne", "solution": "Redémarrer spooler d'impression", "resolution_time": "10min"},
        {"ticket": "Impression bloquée", "solution": "Vider file d'attente", "resolution_time": "5min"}
    ]
}

# Définir les outils que ReAct peut utiliser
def search_similar_tickets(category: str) -> str:
    """Cherche des tickets similaires dans la base de données."""
    category_map = {
        "Hardware": "hardware_boot",
        "Network": "network_vpn",
        "Peripherals": "printer"
    }
    
    key = category_map.get(category, None)
    if key and key in ticket_database:
        tickets = ticket_database[key]
        result = f"Tickets similaires trouvés ({len(tickets)}):\n"
        for i, ticket in enumerate(tickets, 1):
            result += f"{i}. {ticket['ticket']} → {ticket['solution']} (résolu en {ticket['resolution_time']})\n"
        return result
    return "Aucun ticket similaire trouvé."

# Test de l'outil
print("🔧 Outil : search_similar_tickets")
print("\nTest avec catégorie 'Hardware':")
print(search_similar_tickets("Hardware"))

In [ ]:
# Créer un module ReAct avec outils
# Note: ReAct dans DSPy nécessite une configuration spéciale avec les outils
# Voici une démonstration conceptuelle

class TicketWithContext(dspy.Signature):
    """Classifier un ticket IT en utilisant des informations contextuelles."""
    ticket = dspy.InputField(desc="Description du ticket")
    similar_tickets = dspy.InputField(desc="Tickets similaires de la base de données")
    category = dspy.OutputField(desc=f"Catégorie parmi: {', '.join(CATEGORIES)}")
    priority = dspy.OutputField(desc=f"Priorité parmi: {', '.join(PRIORITIES)}")
    suggested_solution = dspy.OutputField(desc="Solution suggérée basée sur les tickets similaires")

# Simuler un workflow ReAct manuel
def react_workflow(ticket_description):
    """Simule un workflow ReAct : Raisonne → Agit → Observe → Répète"""
    
    print("🔄 WORKFLOW REACT")
    print("="*70)
    
    # Étape 1 : Raisonnement initial
    print("\n1️⃣ RAISONNEMENT : Analyser le ticket pour identifier la catégorie")
    classifier = dspy.Predict(TicketClassifier)
    initial_result = classifier(ticket=ticket_description)
    print(f"   → Catégorie identifiée : {initial_result.category}")
    
    # Étape 2 : Action - Chercher des tickets similaires
    print("\n2️⃣ ACTION : Chercher des tickets similaires dans la base")
    similar = search_similar_tickets(initial_result.category)
    print(f"   → {similar.split(chr(10))[0]}")  # Première ligne du résultat
    
    # Étape 3 : Observation - Analyser les résultats
    print("\n3️⃣ OBSERVATION : Utiliser les tickets similaires pour affiner la réponse")
    enhanced_classifier = dspy.ChainOfThought(TicketWithContext)
    final_result = enhanced_classifier(
        ticket=ticket_description,
        similar_tickets=similar
    )
    
    # Étape 4 : Réponse finale
    print("\n4️⃣ RÉSULTAT FINAL")
    print(f"   📦 Catégorie : {final_result.category}")
    print(f"   ⚡ Priorité : {final_result.priority}")
    print(f"   💡 Solution suggérée : {final_result.suggested_solution}")
    
    return final_result

# Tester le workflow ReAct
test_ticket_react = "Mon ordinateur portable ne s'allume plus du tout."
result = react_workflow(test_ticket_react)

## Module 4 : ProgramOfThought (génération de code)

**ProgramOfThought** génère du code Python pour raisonner sur des problèmes complexes, particulièrement utile pour les calculs et la manipulation de données.

### Fonctionnement

1. **Analyse** le problème
2. **Génère** du code Python pour le résoudre
3. **Exécute** le code
4. **Utilise** le résultat pour générer la réponse finale

### Quand l'utiliser

- Problèmes mathématiques
- Calculs complexes
- Manipulation de données structurées
- Tâches nécessitant une logique précise

### ⚠️ Note importante sur cet exemple

L'exemple ci-dessous est une **simulation pédagogique** pour illustrer le concept de ProgramOfThought. Le code est codé en dur plutôt que généré dynamiquement par le modèle.

**Pourquoi une simulation ?**
- ProgramOfThought nécessite un environnement d'exécution Python sécurisé
- La configuration est plus complexe que les autres modules
- Cela permet de comprendre le concept sans les détails techniques

**Utilisation réelle de `dspy.ProgramOfThought` :**

Pour utiliser le vrai module, vous devriez :
1. Définir une signature avec un champ de sortie pour le code
2. Utiliser `dspy.ProgramOfThought(Signature)` ou `dspy.TypedPredictor`
3. Configurer un environnement d'exécution sécurisé (sandbox)
4. Le modèle génère alors automatiquement le code

Cette approche nécessite des considérations de sécurité importantes (injection de code, accès aux ressources, etc.) qui dépassent le cadre de ce tutoriel d'introduction.

### Exemple : Simulation du concept

Voyons comment ProgramOfThought analyserait des statistiques de tickets.

In [ ]:
# ⚠️ SIMULATION PÉDAGOGIQUE ⚠️
# Ce code simule le comportement de ProgramOfThought
# Le code est codé en dur pour illustrer le concept, 
# plutôt que généré dynamiquement par le modèle

class TicketStats(dspy.Signature):
    """Calculer des statistiques sur les temps de résolution de tickets."""
    ticket_data = dspy.InputField(desc="Données des tickets avec temps de résolution")
    question = dspy.InputField(desc="Question statistique à répondre")
    code = dspy.OutputField(desc="Code Python pour calculer la réponse")
    answer = dspy.OutputField(desc="Réponse à la question")

def simulate_program_of_thought():
    """
    Simule comment ProgramOfThought fonctionnerait.
    
    Dans la vraie utilisation :
    - Le modèle recevrait ticket_data et question
    - Il générerait automatiquement le code Python
    - Le code serait exécuté dans un environnement sécurisé
    - Le résultat serait retourné comme réponse
    """
    
    # Données de tickets avec temps de résolution (en minutes)
    tickets_data = """
    Hardware: [120, 60, 90, 45]
    Software: [30, 45, 20, 35]
    Network: [15, 25, 20, 30]
    """
    
    question = "Quelle est la catégorie avec le temps de résolution moyen le plus élevé ?"
    
    print("💻 SIMULATION PROGRAM OF THOUGHT")
    print("="*70)
    print(f"\n📊 Données : {tickets_data.strip()}")
    print(f"❓ Question : {question}\n")
    
    # ⚠️ CODE CODÉ EN DUR (dans la réalité, le modèle le générerait)
    # Ce que ProgramOfThought générerait automatiquement :
    generated_code = """
tickets = {
    'Hardware': [120, 60, 90, 45],
    'Software': [30, 45, 20, 35],
    'Network': [15, 25, 20, 30]
}

# Calculer la moyenne pour chaque catégorie
averages = {cat: sum(times) / len(times) for cat, times in tickets.items()}

# Trouver la catégorie avec la moyenne la plus élevée
max_category = max(averages, key=averages.get)
max_average = averages[max_category]

result = f"{max_category} avec {max_average:.1f} minutes en moyenne"
"""
    
    print("🔧 Code qui serait généré par le modèle :")
    print("-" * 70)
    print(generated_code)
    print("-" * 70)
    
    # Exécution du code (cette partie serait faite par DSPy)
    print("\n⚡ Exécution du code...")
    exec_globals = {}
    exec(generated_code, exec_globals)
    result = exec_globals['result']
    
    print(f"✅ Résultat : {result}\n")
    
    print("💡 Dans la vraie utilisation de dspy.ProgramOfThought :")
    print("   - Le modèle génère le code automatiquement")
    print("   - DSPy l'exécute dans un environnement sécurisé")
    print("   - Vous obtenez directement le résultat")
    
    return result

# Exécuter la simulation
simulate_program_of_thought()

## Module 5 : Modules personnalisés (composition)

Vous pouvez créer vos propres modules en **composant** plusieurs modules existants. C'est l'une des fonctionnalités les plus puissantes de DSPy.

### Pourquoi composer des modules ?

- **Décomposer** une tâche complexe en sous-tâches
- **Réutiliser** des modules existants
- **Créer** des pipelines sophistiqués
- **Améliorer** la précision avec des stratégies avancées

### Comment créer un module personnalisé ?

1. Hériter de `dspy.Module`
2. Initialiser les sous-modules dans `__init__`
3. Définir la logique dans `forward()`
4. Retourner un `dspy.Prediction`

Voyons trois exemples concrets de composition.

### Exemple 1 : Pipeline séquentiel

Classifier d'abord la catégorie, puis la priorité **en fonction** de la catégorie identifiée.

**Avantage** : La deuxième étape peut utiliser le résultat de la première pour affiner sa décision.

In [ ]:
# Définir des signatures spécialisées
class CategoryClassifier(dspy.Signature):
    """Déterminer la catégorie technique d'un ticket IT."""
    ticket = dspy.InputField(desc="Description du ticket")
    category = dspy.OutputField(desc=f"Catégorie parmi: {', '.join(CATEGORIES)}")

class PriorityClassifier(dspy.Signature):
    """Déterminer la priorité d'un ticket en fonction de sa catégorie."""
    ticket = dspy.InputField(desc="Description du ticket")
    category = dspy.InputField(desc="Catégorie technique déjà identifiée")
    priority = dspy.OutputField(desc=f"Priorité parmi: {', '.join(PRIORITIES)}")


# Créer un module composé avec pipeline séquentiel
class SequentialClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        # Initialiser les sous-modules
        self.category_predictor = dspy.ChainOfThought(CategoryClassifier)
        self.priority_predictor = dspy.ChainOfThought(PriorityClassifier)

    def forward(self, ticket):
        # Étape 1 : Prédire la catégorie
        category_result = self.category_predictor(ticket=ticket)

        # Étape 2 : Prédire la priorité en utilisant la catégorie
        priority_result = self.priority_predictor(
            ticket=ticket,
            category=category_result.category
        )

        # Retourner les résultats combinés
        return dspy.Prediction(
            category=category_result.category,
            priority=priority_result.priority
        )


# Tester le module composé
sequential = SequentialClassifier()
result = sequential(ticket=test_ticket)

print("🔗 Module composé : SequentialClassifier")
print(f"📝 Ticket : {test_ticket}")
print(f"📦 Catégorie (étape 1) : {result.category}")
print(f"⚡ Priorité (étape 2, basée sur catégorie) : {result.priority}")

### Exemple 2 : Module avec validation

Ajouter une étape de validation pour vérifier que les prédictions sont valides et les corriger si nécessaire.

**Avantage** : Garantit que les sorties respectent toujours les contraintes définies.

In [ ]:
# Créer un module avec validation
class ValidatedClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        self.classifier = dspy.ChainOfThought(TicketClassifier)
    
    def forward(self, ticket):
        # Prédiction
        result = self.classifier(ticket=ticket)
        
        # Validation de la catégorie
        if result.category not in CATEGORIES:
            print(f"⚠️  Catégorie invalide '{result.category}', correction...")
            result.category = "Application"  # Valeur par défaut
        
        # Validation de la priorité
        if result.priority not in PRIORITIES:
            print(f"⚠️  Priorité invalide '{result.priority}', correction...")
            result.priority = "Medium"  # Valeur par défaut
        
        return result

# Tester le module avec validation
validated = ValidatedClassifier()
result = validated(ticket=test_ticket)

print("\n✅ Module avec validation : ValidatedClassifier")
print(f"📦 Catégorie validée : {result.category}")
print(f"⚡ Priorité validée : {result.priority}")

### Exemple 3 : Module avec consensus (ensemble)

Utiliser plusieurs modules et combiner leurs prédictions par vote majoritaire.

**Avantage** : Améliore la robustesse en réduisant les erreurs aléatoires. Typiquement +3-10% de précision.

In [ ]:
# Créer un module avec consensus (ensemble)
class EnsembleClassifier(dspy.Module):
    def __init__(self):
        super().__init__()
        # Créer plusieurs classifieurs
        self.classifiers = [
            dspy.Predict(TicketClassifier),
            dspy.ChainOfThought(TicketClassifier),
            dspy.ChainOfThought(TicketClassifier)  # 2x ChainOfThought pour plus de poids
        ]

    def forward(self, ticket):
        # Obtenir les prédictions de tous les classifieurs
        predictions = [clf(ticket=ticket) for clf in self.classifiers]

        # Vote majoritaire pour la catégorie
        categories = [p.category for p in predictions]
        category = max(set(categories), key=categories.count)

        # Vote majoritaire pour la priorité
        priorities = [p.priority for p in predictions]
        priority = max(set(priorities), key=priorities.count)

        return dspy.Prediction(
            category=category, 
            priority=priority,
            votes={"categories": categories, "priorities": priorities}
        )


# Tester le module avec consensus
ensemble = EnsembleClassifier()
result = ensemble(ticket=test_ticket)

print("🗳️  Module avec consensus : EnsembleClassifier")
print(f"📝 Ticket : {test_ticket}")
print(f"\n📊 Votes pour la catégorie : {result.votes['categories']}")
print(f"📦 Catégorie consensus : {result.category}")
print(f"\n📊 Votes pour la priorité : {result.votes['priorities']}")
print(f"⚡ Priorité consensus : {result.priority}")

## Bonnes pratiques pour les modules

Pour tirer le meilleur parti des modules DSPy, suivez ces recommandations issues de l'expérience pratique.

### ✅ À faire

1. **Commencer simple** : Utilisez d'abord `Predict`, puis `ChainOfThought` si nécessaire
   - Mesurez l'impact avant de complexifier
   - ChainOfThought coûte environ 2x plus cher en tokens

2. **Nommer clairement** : `TicketClassifier` plutôt que `Classifier1`
   - Les noms explicites facilitent la maintenance
   - Documentez l'intention de chaque module

3. **Un module = une tâche** : Gardez les modules focalisés
   - Respectez le principe de responsabilité unique
   - Facilitez les tests et le débogage

4. **Composer progressivement** : Testez chaque module individuellement
   - Validez chaque composant avant l'intégration
   - Utilisez `inspect_history()` pour vérifier

5. **Documenter** : Ajoutez des docstrings à vos modules personnalisés
   - Expliquez le "pourquoi" de la composition
   - Documentez les dépendances entre modules

### ❌ À éviter

1. **Utiliser ChainOfThought partout**
   - Plus lent (~2x)
   - Plus coûteux en tokens (~2x)
   - Utilisez-le seulement quand la réflexion est nécessaire

2. **Trop de composition**
   - Gardez les pipelines compréhensibles (max 3-4 étapes)
   - Au-delà, envisagez de refactoriser

3. **Oublier la validation**
   - Vérifiez toujours les sorties contre vos contraintes
   - Ajoutez des assertions dans vos modules personnalisés

4. **Ne pas mesurer**
   - Utilisez des métriques pour comparer les modules
   - Mesurez latence, coût et précision
   - Documentez les compromis

### 📊 Comparaison récapitulative

| Module | Complexité | Coût (tokens) | Précision | Cas d'usage |
|--------|-----------|---------------|-----------|-------------|
| **Predict** | ⭐ | 1x | Baseline | Tâches simples, prototypes |
| **ChainOfThought** | ⭐⭐ | ~2x | +5-15% | Raisonnement requis |
| **ReAct** | ⭐⭐⭐ | ~3-5x | +10-20% | Avec outils/actions |
| **ProgramOfThought** | ⭐⭐⭐ | ~2-3x | +15-25% | Calculs, logique précise |
| **Modules composés** | ⭐⭐⭐⭐ | Variable | Variable | Pipelines complexes |

### 💡 Conseil final

Commencez toujours par la solution la plus simple qui pourrait fonctionner. DSPy est conçu pour vous permettre d'itérer rapidement :

1. Prototype avec `Predict`
2. Mesurez la performance de base
3. Ajoutez de la complexité si nécessaire
4. Optimisez avec les techniques avancées (voir notebooks suivants)

La vraie puissance de DSPy vient de sa capacité à **optimiser automatiquement** vos modules, ce que nous verrons dans les prochaines parties du tutoriel.

## Inspecter et déboguer les modules

Lorsque vous travaillez avec des modules DSPy, il est souvent utile de voir **exactement ce qui est envoyé au modèle de langage**. C'est là qu'intervient `dspy.inspect_history()`.

### Pourquoi inspecter l'historique ?

- 🔍 **Comprendre** ce que fait réellement chaque module sous le capot
- 🐛 **Déboguer** quand les résultats ne sont pas ceux attendus
- 📊 **Comparer** les prompts générés par différents modules
- 🎓 **Apprendre** comment DSPy construit ses prompts

### Utilisation de base

`dspy.inspect_history(n=3)` affiche les `n` derniers appels au modèle de langage, avec :
- Les **prompts** envoyés au modèle
- Les **réponses** reçues du modèle
- Les **métadonnées** (température, modèle utilisé, etc.)

In [ ]:
# Reconfigurer avec llama3.1 pour les exemples suivants
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)
dspy.configure(lm=lm)

# Faire quelques appels pour avoir de l'historique
predict_classifier = dspy.Predict(TicketClassifier)
cot_classifier = dspy.ChainOfThought(TicketClassifier)

# Appel 1 : Predict
ticket1 = "L'imprimante du 3ème étage ne fonctionne plus."
result1 = predict_classifier(ticket=ticket1)

# Appel 2 : ChainOfThought
ticket2 = "Le VPN se déconnecte toutes les 5 minutes. J'ai un deadline ce soir."
result2 = cot_classifier(ticket=ticket2)

print("✅ Deux appels effectués (Predict + ChainOfThought)")
print("   Utilisez dspy.inspect_history() pour voir les détails")

In [ ]:
# Inspecter les 2 derniers appels
dspy.inspect_history(n=2)

### Que voyons-nous dans l'historique ?

Pour chaque appel, `inspect_history()` affiche :

1. **Le prompt système** : Instructions données au modèle
2. **Le prompt utilisateur** : Les données d'entrée formatées
3. **La réponse** : Ce que le modèle a généré
4. **Les métadonnées** : Température, modèle, tokens utilisés

### Différences clés entre Predict et ChainOfThought

En inspectant l'historique, vous remarquerez que :

**Predict** :
- Prompt direct : "Voici le ticket, donne-moi catégorie et priorité"
- Pas de champ de raisonnement dans le prompt
- Réponse immédiate

**ChainOfThought** :
- Prompt enrichi : Demande d'abord un raisonnement
- Champ `rationale` ajouté dans le prompt
- Le modèle doit d'abord expliquer sa réflexion
- Puis générer la réponse finale

### Cas d'usage pratiques

`dspy.inspect_history()` est particulièrement utile pour :

- **Optimisation** : Comprendre comment ajuster vos signatures
- **Débogage** : Identifier pourquoi un module donne de mauvais résultats
- **Formation** : Voir comment les modules transforment vos signatures en prompts
- **Comparaison** : Évaluer l'impact de différents modules sur le même problème

In [ ]:
# Accéder programmatiquement à l'historique pour comparaison
history = lm.history

print("="*70)
print("Comparaison des prompts : Predict vs ChainOfThought")
print("="*70 + "\n")

if len(history) >= 2:
    # Premier appel (Predict)
    print("🔮 PREDICT")
    print("-" * 70)
    predict_call = history[-2]
    print(f"Messages envoyés : {len(predict_call['messages'])} message(s)")
    print(f"Ticket : {ticket1}")
    print(f"Résultat : {result1.category} | {result1.priority}\n")
    
    # Deuxième appel (ChainOfThought)
    print("🧠 CHAINOFTHOUGHT")
    print("-" * 70)
    cot_call = history[-1]
    print(f"Messages envoyés : {len(cot_call['messages'])} message(s)")
    print(f"Ticket : {ticket2}")
    print(f"Résultat : {result2.category} | {result2.priority}")
    
    # Vérifier si ChainOfThought a généré un raisonnement
    if hasattr(result2, 'rationale'):
        print(f"Raisonnement : {result2.rationale}")
else:
    print("⚠️ Historique insuffisant. Exécutez d'abord les cellules précédentes.")

## Ce qu'est `/no_think`

Certains modèles ont une capacité unique de basculer entre un mode "thinking" (pour le raisonnement logique complexe) et un mode "non-thinking" (pour le dialogue général efficace).

Par exemple, Qwen3 génère par défaut du contenu de réflexion enveloppé dans un bloc `<think>...</think>`, suivi de la réponse finale.

Pour contrôler le mode thinking dans Qwen3, une [suggestion sur Discord](https://discord.com/channels/1161519468141355160/1424016693390217299/1426772883903873177) propose d'ajouter un champ d'entrée dans la signature DSPy.

In [11]:
import dspy
import warnings
warnings.filterwarnings('ignore')

# Configurer le modèle de langage
lm = dspy.LM(
    model='ollama_chat/qwen3:latest',
    api_base='http://localhost:11434',
    temperature=0.3
)

# Configurer DSPy globalement
dspy.configure(lm=lm)

In [13]:
# Créer un module Predict avec notre signature
predict_classifier = dspy.Predict(TicketClassifier)

# Tester sur un exemple
test_ticket = "Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure."
result = predict_classifier(no_think="/no_think", ticket=test_ticket)

print("🔮 Module : Predict")
print(f"📝 Ticket : {test_ticket}")
print(f"📦 Catégorie prédite : {result.category}")
print(f"⚡ Priorité prédite : {result.priority}")

🔮 Module : Predict
📝 Ticket : Mon ordinateur ne démarre plus. J'ai une présentation dans 1 heure.
📦 Catégorie prédite : Hardware
⚡ Priorité prédite : High


## Récapitulatif

Dans ce notebook, nous avons exploré les **modules DSPy**, les composants qui exécutent vos signatures.

### Ce que vous avez appris

1. **Configuration des modèles** : Ollama (local) et Claude (API)
2. **Module Predict** : Génération directe, simple et rapide
3. **Module ChainOfThought** : Raisonnement avant réponse
4. **Module ReAct** : Interaction avec des outils externes
5. **Module ProgramOfThought** : Génération de code pour calculs
6. **Modules personnalisés** : Composition pour pipelines complexes
7. **Inspection et débogage** : `dspy.inspect_history()`
8. **Bonnes pratiques** : Comment choisir et utiliser les modules

### Points clés à retenir

- Les modules transforment vos **signatures** en **prompts optimisés**
- Commencez simple (`Predict`) et complexifiez au besoin
- La composition permet de résoudre des problèmes complexes
- Toujours mesurer avant d'optimiser
- `inspect_history()` est votre meilleur ami pour déboguer

### Prochaines étapes

Dans le **notebook 03-evaluation**, nous verrons comment :
- Définir des métriques pour évaluer vos modules
- Créer des jeux de données de validation
- Comparer objectivement différents modules
- Préparer le terrain pour l'optimisation automatique

Les modules sont puissants, mais leur vraie force se révèle quand on les évalue et les optimise. C'est ce que nous allons découvrir ensuite ! 🚀